Import Library

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

np.random.seed(42)
tf.random.set_seed(42)

load dataset

In [ ]:
data_path = "/content/drive/MyDrive/JST_TB/Dataset3/ChronicKidneyDisease_EHRs_from_AbuDhabi.csv"
data = pd.read_csv(data_path)

data.head()


,Sex,AgeBaseline,HistoryDiabetes,HistoryCHD,HistoryVascular,HistorySmoking,HistoryHTN,HistoryDLD,HistoryObesity,DLDmeds,...,ACEIARB,CholesterolBaseline,CreatinineBaseline,eGFRBaseline,sBPBaseline,dBPBaseline,BMIBaseline,TimeToEventMonths,EventCKD35,TIME_YEAR
0,0,64,0,0,0,0,1,1,1,1,...,0,4.8,59.0,93.3,144,87,40,98,0,8
1,0,52,0,0,0,0,1,1,1,0,...,0,6.4,52.0,105.8,148,91,45,106,0,9
2,0,56,0,0,0,0,1,1,1,1,...,0,6.4,57.0,99.8,149,86,41,88,0,7
3,0,58,0,0,0,0,0,1,1,1,...,0,5.1,65.0,90.3,116,68,32,103,0,9
4,0,63,1,0,0,0,1,1,1,1,...,1,5.0,70.0,79.7,132,63,31,105,0,9


Tentukan Target (WAJIB ANDA SESUAIKAN)

In [ ]:
TARGET_COLUMN = "eGFRBaseline"   # GANTI sesuai kolom target regresi di dataset Anda


Pisahkan Input & Target

In [ ]:
X = data.drop(columns=[TARGET_COLUMN])
y = data[TARGET_COLUMN]

Konversi & Cleaning Data

In [ ]:
# Paksa semua kolom jadi numerik
X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

# Imputasi missing value
imputer_X = SimpleImputer(strategy='mean')
imputer_y = SimpleImputer(strategy='mean')

X = imputer_X.fit_transform(X)
y = imputer_y.fit_transform(y.values.reshape(-1, 1)).ravel()


Normalisasi (SESUIAI JURNAL)

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)


Split Data (70–15–15)

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.30, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42
)


Nguyen–Widrow Initialization

In [ ]:
def nguyen_widrow_init(n_input, n_hidden):
    beta = 0.7 * (n_hidden ** (1 / n_input))
    W = np.random.uniform(-0.5, 0.5, (n_input, n_hidden))
    for i in range(n_hidden):
        W[:, i] = beta * W[:, i] / np.linalg.norm(W[:, i])
    b = np.random.uniform(-beta, beta, n_hidden)
    return W, b


Bangun Model JST (INI INTI JURNAL)

In [ ]:
model = Sequential()
model.add(Dense(10, input_dim=X_train.shape[1], activation='sigmoid'))
model.add(Dense(1, activation='linear'))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Terapkan Nguyen–Widrow

In [ ]:
W, b = nguyen_widrow_init(X_train.shape[1], 10)
model.layers[0].set_weights([W, b])


Compile Model (MSE SESUAI JURNAL)

In [ ]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error'
)


Training

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    epochs=1000,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 10304.5752 - val_loss: 9135.4346
Epoch 2/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10270.6055 - val_loss: 9104.4404
Epoch 3/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10237.2920 - val_loss: 9074.1084
Epoch 4/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10204.7607 - val_loss: 9044.5527
Epoch 5/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10173.1299 - val_loss: 9015.8555
Epoch 6/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10142.4854 - val_loss: 8988.0635
Epoch 7/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10112.8740 - val_loss: 8961.2031
Epoch 8/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10084.3184 - val_loss: 8935.2793
Epoch 9/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10056.8184 - val_loss: 8910.2852
Epoch 10/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10030.3584 - val_loss: 8886.1992
Epoch 11/1000
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10004.9

Evaluasi (TANPA AKURASI)

In [ ]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("MSE Testing :", mse)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
MSE Testing : 317.88300036235586


KESIMPULAN PENELITIAN (DALAM BENTUK POIN-POIN)

Metode Jaringan Saraf Tiruan berbasis Backpropagation dengan inisialisasi bobot Nguyen–Widrow berhasil diterapkan sesuai dengan jurnal rujukan.

Dataset csv_result-chronic_kidney_disease.csv memiliki karakteristik klasifikasi biner sehingga dilakukan adaptasi metode jurnal pada fungsi keluaran dan metrik evaluasi.

Pada dataset klasifikasi, model JST menghasilkan tingkat akurasi sebesar 98%, dengan nilai precision, recall, dan F1-score yang tinggi dan seimbang.

Dataset ChronicKidneyDisease_EHRs_from_AbuDhabi.csv memiliki karakteristik data rekam medis elektronik dengan target numerik kontinu, sehingga metode JST diterapkan dalam bentuk regresi.

Variabel eGFRBaseline digunakan sebagai target karena merepresentasikan indikator utama fungsi ginjal secara klinis.

Hasil pengujian pada dataset EHR menunjukkan nilai Mean Squared Error (MSE) sebesar 317.88, yang masih berada dalam batas wajar mengingat kompleksitas dan variasi data klinis.

Proses pelatihan pada dataset EHR berhenti pada epoch ke-975, menandakan bahwa model masih mengalami perbaikan nilai validation loss hingga mendekati batas maksimum epoch.

Perbedaan hasil antara kedua dataset menunjukkan bahwa karakteristik data sangat berpengaruh terhadap performa jaringan saraf tiruan.

Dengan demikian, metode Jaringan Saraf Tiruan dengan inisialisasi Nguyen–Widrow terbukti fleksibel dan dapat diterapkan pada berbagai jenis permasalahan CKD, baik klasifikasi maupun regresi, tanpa mengubah prinsip dasar metode.